In [0]:
# Databricks-ready PySpark code for dataset: ott_user_behavior_1
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("OTT_Analytics").getOrCreate()

# ---------- 1) Load dataset (try Spark table, else CSV fallback) ----------
dataset_name = "ott_user_behavior_1"

try:
    df = spark.table(dataset_name)
    print(f"Loaded Spark table: {dataset_name}")
except Exception as e:
    print(f"Table {dataset_name} not found, trying CSV at /FileStore/tables/{dataset_name}.csv")
    df = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"/FileStore/tables/{dataset_name}.csv")

# quick clean / cast commonly-needed columns
# adjust column names if your schema differs (case-sensitive in Spark)
# Ensure numeric columns are numeric
num_cols = ["age", "avg_watch_time_per_day", "total_watch_time", "watch_duration",
            "completion_rate", "rating_given", "total_sessions_per_week",
            "avg_stream_quality", "network_speed_mbps", "buffering_count",
            "monthly_spend", "peak_watch_hour", "churn_flag"]
for c in num_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(DoubleType()))

# Fill NA for numeric cols with 0 for safe model training (simple strategy)
numeric_present = [c for c in num_cols if c in df.columns]
df = df.fillna({c: 0.0 for c in numeric_present})

Loaded Spark table: ott_user_behavior_1


In [0]:
from pyspark.sql.functions import col, lit, least, greatest, explode
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS

# ✅ Load directly from catalog, not CSV
df = spark.table("workspace.default.ott_user_behavior_1")

In [0]:
# ---------- USE CASE 4: REGIONAL CONTENT DEMAND FORECASTING ----------

from pyspark.sql.functions import (
    col, to_timestamp, date_format, sum as _sum, avg, countDistinct, round, desc
)
from pyspark.sql import functions as F

# ✅ Load data
df = spark.table("workspace.default.ott_user_behavior_1")

# ---------- MAIN AGGREGATION ----------
if "watch_date" in df.columns and "genre" in df.columns:
    # Convert to timestamp and extract month
    df_dates = df.withColumn("watch_ts", to_timestamp(col("watch_date"), "dd-MM-yyyy HH:mm").cast("timestamp"))
    df_dates = df_dates.withColumn("watch_month", date_format(col("watch_ts"), "yyyy-MM"))

    # Regional + Genre monthly summary
    regional_demand = (
        df_dates.groupBy("region", "genre", "watch_month")
        .agg(
            round(_sum("total_watch_time"), 2).alias("sum_watch_time"),
            round(countDistinct("user_id"), 0).alias("unique_users"),
            round(avg("rating_given"), 2).alias("avg_rating")
        )
        .orderBy("region", "genre", "watch_month")
    )

    display(regional_demand.limit(200))

else:
    # Fallback if no date field
    regional_demand = (
        df.groupBy("region", "genre")
        .agg(
            round(_sum("total_watch_time"), 2).alias("sum_watch_time"),
            round(avg("rating_given"), 2).alias("avg_rating"),
            countDistinct("user_id").alias("unique_users")
        )
        .orderBy(desc("sum_watch_time"))
    )
    display(regional_demand)


# ---------- ADDITIONAL VISUALS ----------

# 1️⃣ Regional total watch time (overall demand)
region_summary = (
    regional_demand.groupBy("region")
    .agg(round(_sum("sum_watch_time"), 2).alias("total_watch_time"))
    .orderBy(desc("total_watch_time"))
)
display(region_summary)

# 2️⃣ Top 10 genres per region (who watches what the most)
top_genres = (
    regional_demand.groupBy("region", "genre")
    .agg(round(_sum("sum_watch_time"), 2).alias("genre_watch_time"))
    .orderBy("region", desc("genre_watch_time"))
)
display(top_genres.limit(30))

# 3️⃣ Monthly demand trend (time series)
if "watch_month" in regional_demand.columns:
    monthly_trend = (
        regional_demand.groupBy("watch_month")
        .agg(
            round(_sum("sum_watch_time"), 2).alias("total_watch_time"),
            round(avg("avg_rating"), 2).alias("avg_rating")
        )
        .orderBy("watch_month")
    )
    display(monthly_trend)

# 4️⃣ Genre performance by average rating
genre_rating = (
    regional_demand.groupBy("genre")
    .agg(round(avg("avg_rating"), 2).alias("genre_avg_rating"))
    .orderBy(desc("genre_avg_rating"))
)
display(genre_rating)

# 5️⃣ Heatmap-style summary: Region × Genre
region_genre_heatmap = (
    regional_demand.groupBy("region", "genre")
    .agg(round(avg("sum_watch_time"), 2).alias("avg_watch_time"))
)
display(region_genre_heatmap)

# ---------- OPTIONAL: SAVE FOR DASHBOARD ----------
regional_demand.write.mode("overwrite").format("delta").saveAsTable("ott_regional_demand_summary")
region_summary.write.mode("overwrite").format("delta").saveAsTable("ott_region_total_demand")
top_genres.write.mode("overwrite").format("delta").saveAsTable("ott_top_genres_by_region")
genre_rating.write.mode("overwrite").format("delta").saveAsTable("ott_genre_rating_summary")

print("✅ Use Case 4 visuals and summary tables created successfully!")

region,genre,watch_month,sum_watch_time,unique_users,avg_rating
Bihar,Action,2025-05,57892.62,10,2.48
Bihar,Action,2025-06,73159.25,11,2.95
Bihar,Action,2025-07,78374.79,13,2.82
Bihar,Action,2025-08,38603.94,8,3.09
Bihar,Action,2025-09,40744.7,5,3.86
Bihar,Action,2025-10,85210.47,13,3.52
Bihar,Action,2025-11,29466.52,6,2.25
Bihar,Comedy,2025-05,52392.3,10,2.68
Bihar,Comedy,2025-06,90486.95,18,2.76
Bihar,Comedy,2025-07,98758.57,17,3.25


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

region,total_watch_time
Bihar,3789604.29
Kerala,3729982.3
Karnataka,3658600.64
Tamil Nadu,3591678.99
Delhi,3568774.64
Maharashtra,3492702.73
Uttar Pradesh,3471244.44
West Bengal,3375899.93


Databricks visualization. Run in Databricks to view.

...

genre,genre_avg_rating
Sci-Fi,3.04
Drama,3.01
Thriller,3.0
Romance,2.99
Horror,2.99
Action,2.97
Documentary,2.93
Comedy,2.92


Databricks visualization. Run in Databricks to view.

region,genre,avg_watch_time
Bihar,Action,57636.04
Bihar,Comedy,69187.4
Bihar,Documentary,58728.8
Bihar,Drama,64383.67
Bihar,Horror,80371.18
Bihar,Romance,80383.36
Bihar,Sci-Fi,62025.28
Bihar,Thriller,68656.32
Delhi,Action,69397.76
Delhi,Comedy,72013.18


Databricks visualization. Run in Databricks to view.

✅ Use Case 4 visuals and summary tables created successfully!
